In [1]:
import requests
import pandas as pd
#import emissionsimpacts as ei
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option("mode.chained_assignment", None)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import random
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap

# load plot setting parameters
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams.update({'font.size': 17.5})
# plt.rcParams['axes.titlesize'] = 18
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16
plt.rcParams['axes.labelpad'] = 10
plt.rcParams['axes.titlepad'] = 15

# do this for one ISO first and then repeat results across all regions

In [2]:
# Load the datasets
CISO = pd.read_csv(
    'data/processed/CISO_merged_D.csv',
    parse_dates=["timestamp"],
    skiprows=0,
)

CISO_ext = pd.read_csv(
    'data/processed/CISO_control.csv',
    parse_dates=["Local time"],
    skiprows=0,
)

# Process CISO_ext
CISO_ext = CISO_ext.rename(columns={"Local time": "timestamp"})

# Instead of summing directly, first select the columns you want to sum, excluding datetime columns
columns_to_sum = [col for col in CISO_ext.columns if col != 'timestamp']

# Perform the grouping and sum only on the selected columns
CISO_ext_D = CISO_ext.groupby(CISO_ext.timestamp.dt.date)[columns_to_sum].sum().reset_index()

# Convert the grouped 'date' back to a datetime format and rename it to 'timestamp'
CISO_ext_D['timestamp'] = pd.to_datetime(CISO_ext_D['timestamp'])

# Merge datasets
CISO.set_index('timestamp', inplace=True)
CISO = CISO.merge(CISO_ext_D, on="timestamp", how="left")

# Add date-related columns and drop NaNs
CISO["month"] = CISO.timestamp.dt.month
CISO["year"] = CISO.timestamp.dt.year
CISO["day"] = CISO.timestamp.dt.day
CISO.dropna(inplace=True)

# Filter out rows with 0 or negative values in specified columns
positive_value_columns = ["solar", "wind", "SUN_ext", "WND_ext", "D_ext", "Wramp"]
for column in positive_value_columns:
    CISO = CISO[CISO[column] > 0]

# Calculate residual demand
CISO["residual_demand"] = CISO.demand + CISO.imports - CISO.hydro

# Function to log-transform specified columns

In [3]:
def log_df_RD(df: pd.DataFrame):
    # Remove any 0s from all columns used below
    df = df.replace(0, np.nan).dropna()
    # Remove inf
    df = df.replace([np.inf, -np.inf], np.nan).dropna()

    # take absolute value of imports

    df["imports"] = df["imports"].abs()

    log_transform_columns = ["generation", "demand", "imports", "hydro", "solar", "wind", "SUN_ext", "WND_ext", "D_ext", "Wramp", "co2", "intensity"]
    for column in log_transform_columns:
        df[column] = np.log(df[column])

    return df

# OLS regression function
def ols_regression_RD(df: pd.DataFrame, variable, log=False):
    if log:
        df = log_df_RD(df)
    model = smf.ols(
        formula=f"{variable} ~ demand + imports + hydro + solar + wind + SUN_ext + WND_ext + D_ext + Wramp + C(id) + C(month) + C(year) + C(id)*C(month)",
        data=df,
    ).fit()
    return model

In [9]:
M1 = ols_regression_RD(CISO, "generation", log=True)
# print coefficient, significance and standard error for demand, solar, wind, SUN_ext, WND_ext
print(M1.params[["demand", "solar", "wind", "Wramp", "SUN_ext", "WND_ext"]])
print(M1.pvalues[["demand", "solar", "wind", "Wramp",  "SUN_ext", "WND_ext"]])
print(M1.bse[["demand", "solar", "wind", "Wramp", "SUN_ext", "WND_ext"]])

demand     2.800126
solar     -0.224362
wind      -0.203615
Wramp      0.118562
SUN_ext    0.003813
WND_ext   -0.018473
dtype: float64
demand     0.000000e+00
solar      4.934990e-13
wind       1.320296e-62
Wramp      1.308440e-15
SUN_ext    9.028718e-01
WND_ext    1.305352e-04
dtype: float64
demand     0.067967
solar      0.031035
wind       0.012173
Wramp      0.014826
SUN_ext    0.031249
WND_ext    0.004828
dtype: float64


In [77]:
M2 = ols_regression_RD(CISO, "co2", log=True)
# print coefficient, significance and standard error for demand, solar, wind, SUN_ext, WND_ext
print(M2.params[["demand", "solar", "wind", "Wramp", "SUN_ext", "WND_ext"]])
print(M2.pvalues[["demand", "solar", "wind", "Wramp", "SUN_ext", "WND_ext"]])
print(M2.bse[["demand", "solar", "wind", "Wramp", "SUN_ext", "WND_ext"]])

demand     1.711479
solar     -0.028284
wind      -0.295196
Wramp      0.105513
SUN_ext    0.012966
WND_ext   -0.030759
dtype: float64
demand     0.000000e+00
solar      3.121088e-04
wind       0.000000e+00
Wramp      2.726359e-33
SUN_ext    5.862359e-02
WND_ext    2.135474e-05
dtype: float64
demand     0.039992
solar      0.007845
wind       0.006927
Wramp      0.008773
SUN_ext    0.006857
WND_ext    0.007237
dtype: float64


In [78]:
M3 = ols_regression_RD(CISO, "intensity", log=True)
# print coefficient, significance and standard error for demand, solar, wind, SUN_ext, WND_ext
print(M3.params[["demand", "solar", "wind", "Wramp", "SUN_ext", "WND_ext"]])
print(M3.pvalues[["demand", "solar", "wind", "Wramp", "SUN_ext", "WND_ext"]])
print(M3.bse[["demand", "solar", "wind", "Wramp", "SUN_ext", "WND_ext"]])

demand    -0.106952
solar      0.000142
wind       0.023603
Wramp     -0.010465
SUN_ext   -0.000004
WND_ext   -0.001479
dtype: float64
demand      1.124378e-63
solar       9.092476e-01
wind       4.328148e-102
Wramp       5.506488e-14
SUN_ext     9.969138e-01
WND_ext     1.975933e-01
dtype: float64
demand     0.006344
solar      0.001244
wind       0.001099
Wramp      0.001392
SUN_ext    0.001088
WND_ext    0.001148
dtype: float64


In [79]:
# print r squared and number of observations
print(M3.rsquared, M1.nobs)

0.9217185594776084 87089.0


In [4]:
df = CISO.copy()
df = log_df_RD(df)

M1 = smf.ols( # original
        formula="generation ~ residual_demand + solar + wind + SUN_ext + WND_ext + D_ext + Wramp + C(id) + C(month) + C(year) + C(id)*C(month)",
        data=df,
    ).fit()

M2 = smf.ols( # exclude residual_demand
        formula="generation ~ solar + wind + SUN_ext + WND_ext + D_ext + Wramp + C(id) + C(month) + C(year) + C(id)*C(month)",
        data=df,
    ).fit()

M3 = smf.ols( # exclude solar
        formula="generation ~ residual_demand + wind + SUN_ext + WND_ext + D_ext + Wramp + C(id) + C(month) + C(year) + C(id)*C(month)",
        data=df,
    ).fit()

M4 = smf.ols( # exclude wind
        formula="generation ~ residual_demand + solar + SUN_ext + WND_ext + D_ext + Wramp + C(id) + C(month) + C(year) + C(id)*C(month)",
        data=df,
    ).fit()

M5 = smf.ols( # exclude external variables
        formula="generation ~ residual_demand + solar + wind + Wramp + C(id) + C(month) + C(year) + C(id)*C(month)",
        data=df,
    ).fit()

M6 = smf.ols( # demand
        formula="generation ~ demand + solar + wind + SUN_ext + WND_ext + D_ext + Wramp + C(id) + C(month) + C(year) + C(id)*C(month)",
        data=df,
    ).fit()

M7 = smf.ols( # exclude wind ramp
        formula="generation ~ residual_demand + solar + wind + SUN_ext + WND_ext + D_ext + C(id) + C(month) + C(year) + C(id)*C(month)",
        data=df,
    ).fit()

M8 = smf.ols( # exclude time fixed effects
        formula="generation ~ residual_demand + solar + wind + SUN_ext + WND_ext + D_ext + Wramp + C(id) + C(id)*C(month)",
        data=df,
    ).fit()

M9 = smf.ols( # exclude interaction
        formula="generation ~ residual_demand + solar + wind + SUN_ext + WND_ext + D_ext + Wramp + C(id) + C(month) + C(year)",
        data=df,
    ).fit()

In [5]:
# create list of residual_demand + solar + wind + SUN_ext + WND_ext + D_ext + Wramp and demand

list = ["residual_demand", "solar", "wind", "SUN_ext", "WND_ext", "D_ext", "Wramp", "demand"]

In [8]:
for variable in list:
    # if key error, then print "Not found"
    try:
        print(M1.params[variable])
    except KeyError:
        print("Not found")

# also print the R-squared values, p-values and std errors

print("----------------R squared----------------")

print("R - ",M1.rsquared)

print("----------------Standard error----------------")

for variable in list:
    # if key error, then print "Not found"
    try:
        print(M1.bse[variable])
    except KeyError:
        print("Not found")

# show p values as stars *** if < 0.01, ** if < 0.05, * if < 0.1

print("----------------P values----------------")

for variable in list:
    # if key error, then print "Not found"
    try:
        if M1.pvalues[variable] < 0.01:
            print("***")
        elif M1.pvalues[variable] < 0.05:
            print("**")
        elif M1.pvalues[variable] < 0.1:
            print("*")
        else:
            print("Not significant")
    except KeyError:
        print("Not found")

4.847878699379993e-06
-0.24213896355731426
-0.21816184442765119
-0.006618971286478217
-0.005991279303733575
0.037768517936144705
0.11184535268261929
Not found
----------------R squared----------------
R -  0.8417373761224349
----------------Standard error----------------
1.022116295612962e-07
0.03094283879948687
0.01150381191440778
0.031009249850621336
0.004678520265602592
0.03251160896698784
0.014708553814964817
Not found
----------------P values----------------
***
***
***
Not significant
Not significant
Not significant
***
Not found


In [7]:
models = [M1, M2, M3, M4, M5, M6, M7, M8, M9]  # Your model instances
variables = ['residual_demand', 'solar', 'wind', 'SUN_ext', 'WND_ext', 'D_ext', 'Wramp', 'demand']  # Your variables

# Function to format coefficient with significance
def format_coefficient(coef, pvalue):
    if pvalue < 0.01:
        return f"{coef:.2f}***"
    elif pvalue < 0.05:
        return f"{coef:.2f}**"
    elif pvalue < 0.1:
        return f"{coef:.2f}*"
    else:
        return f"{coef:.2f}"

# Start building the LaTeX table
latex_table = """
\\begin{table*}[ht!]
\\centering
\\label{table:coefficients}
\\begin{tabular}{lccccccccc}
    \\toprule
    \\rowcolor{Gray} Variable & M1 & M2 & M3 & M4 & M5 & M6 & M7 & M8 & M9 \\\\
    \\midrule
"""

# Loop over each variable and model to populate coefficients and standard errors
for variable in variables:
    coef_row = [variable]  # Coefficient row starts with the variable name
    se_row = [""]  # Standard error row starts empty for the variable name column
    for model in models:
        try:
            coef = model.params[variable]
            pvalue = model.pvalues[variable]
            std_err = model.bse[variable]
            formatted_coef = format_coefficient(coef, pvalue)
            coef_row.append(formatted_coef)
            se_row.append(f"({std_err:.2f})")  # Standard errors formatted
        except KeyError:
            coef_row.append("-")  # Coefficient not found
            se_row.append("-")  # Standard error not found
    latex_table += "    " + " & ".join(coef_row) + " \\\\\n"
    latex_table += "    " + " & ".join(se_row) + " \\\\\n"

# Add R-squared row
rsquared_row = ["R-squared"] + [f"{model.rsquared:.2f}" for model in models]
latex_table += "    \\midrule\n"
latex_table += "    " + " & ".join(rsquared_row) + " \\\\\n"

# Add fixed elements like Time FE and Interaction flags
latex_table += """
    Time FE & Y & Y & Y & Y & Y & Y & Y & Y & Y \\\\
    Interaction & Y & Y & Y & Y & Y & Y & Y & Y & Y \\\\
    \\bottomrule
\\end{tabular}
    \\caption{Coefficients for the panel regression formulation performed on hourly data aggregated to daily time series for various model specifications.}
\\end{table*}
"""

print(latex_table)



\begin{table*}[ht!]
\centering
\label{table:coefficients}
\begin{tabular}{lccccccccc}
    \toprule
    \rowcolor{Gray} Variable & M1 & M2 & M3 & M4 & M5 & M6 & M7 & M8 & M9 \\
    \midrule
    residual_demand & 0.00*** & - & 0.00*** & 0.00*** & 0.00*** & - & 0.00*** & 0.00*** & 0.00*** \\
     & (0.00) & - & (0.00) & (0.00) & (0.00) & - & (0.00) & (0.00) & (0.00) \\
    solar & -0.24*** & -0.06** & - & -0.21*** & -0.25*** & -0.14*** & -0.25*** & -0.23*** & -0.23*** \\
     & (0.03) & (0.03) & - & (0.03) & (0.02) & (0.03) & (0.03) & (0.03) & (0.03) \\
    wind & -0.22*** & -0.23*** & -0.21*** & - & -0.22*** & -0.13*** & -0.15*** & -0.23*** & -0.21*** \\
     & (0.01) & (0.01) & (0.01) & - & (0.01) & (0.01) & (0.01) & (0.01) & (0.01) \\
    SUN_ext & -0.01 & -0.15*** & -0.19*** & 0.00 & - & 0.02 & -0.00 & -0.07** & -0.02 \\
     & (0.03) & (0.03) & (0.02) & (0.03) & - & (0.03) & (0.03) & (0.03) & (0.03) \\
    WND_ext & -0.01 & -0.06*** & -0.01** & -0.02*** & - & -0.06*** & -0.00 & -0.0